# Loading images and matching them to metadata

In [1]:
#@title Imports and function definitions


# For running inference on the TF-Hub module.
import tensorflow as tf

import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time

# Print Tensorflow version
print(tf.__version__)

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

2022-04-12 14:23:35.996959: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


2.4.1
The following GPU devices are available: /device:GPU:0


2022-04-12 14:23:39.231607: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-12 14:23:39.236573: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-04-12 14:23:39.559329: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:04:00.0 name: NVIDIA GeForce GTX TITAN X computeCapability: 5.2
coreClock: 1.076GHz coreCount: 24 deviceMemorySize: 11.93GiB deviceMemoryBandwidth: 313.37GiB/s
2022-04-12 14:23:39.560162: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:83:00.0 name: NVIDIA GeForce GTX TITAN X computeCapability: 5.2
coreClock

In [2]:
# adds scripts/ and src/ folder: so you can import scripts/functions across project steps
import sys 
sys.path.append("../../src")
sys.path.append("../../scripts")

from os import listdir
from os import path

import pandas as pd
from tqdm.notebook import tqdm

from data_filepaths import image_folders, portraits_csv, images_with_boxes_folder, object_detection_results_path

 GPU (device: 1, name: NVIDIA GeForce GTX TITAN X, pci bus id: 0000:83:00.0, compute capability: 5.2)
2022-04-12 14:23:41.273413: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


## Linking portraits' metadata with their path

#### Loading portraits' metadata

In [3]:
portraits = pd.read_csv(portraits_csv)
portraits.head()

,Unnamed: 0,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename,correcteddate
0,70345,Masaccio,1425,portrait,548.0,800.0,116190.0,wikiart,Early Renaissance,Portrait of a Young Man,train_only,True,34161.jpg,1425.0
1,20989,Masaccio,1425,portrait,999.0,1300.0,899549.0,wikiart,Early Renaissance,Portrait of a Young Woman,train_only,True,80100.jpg,1425.0
2,12749,Robert Campin,c.1430,portrait,1256.0,1857.0,219654.0,wikiart,Northern Renaissance,Portrait of a Woman,test_only,False,85550.jpg,1430.0
3,2904,Robert Campin,c.1430,portrait,2024.0,3019.0,416418.0,wikiart,Northern Renaissance,Portrait of a Man,test_only,False,2662.jpg,1430.0
4,47846,Jan van Eyck,1432,portrait,502.0,1451.0,137626.0,wikiart,Northern Renaissance,"Isabella Borluut, panel from the Ghent Altarpiece",train_and_test,True,66905.jpg,1432.0


#### Getting images paths

In [4]:
image_folder = image_folders[0]
train0 = listdir(image_folder)
train0 = train0#[0:10] # SMALL SELECTION

image_paths = pd.DataFrame({
    "new_filename":train0,
    "path": [path.join(image_folder, n) for n in train0]
})

image_paths

,new_filename,path
0,93571.jpg,../../data/kaggle_painters_by_numbers/train/93...
1,90123.jpg,../../data/kaggle_painters_by_numbers/train/90...
2,71660.jpg,../../data/kaggle_painters_by_numbers/train/71...
3,11898.jpg,../../data/kaggle_painters_by_numbers/train/11...
4,73354.jpg,../../data/kaggle_painters_by_numbers/train/73...
...,...,...
79428,46349.jpg,../../data/kaggle_painters_by_numbers/train/46...
79429,10051.jpg,../../data/kaggle_painters_by_numbers/train/10...
79430,12831.jpg,../../data/kaggle_painters_by_numbers/train/12...
79431,12678.jpg,../../data/kaggle_painters_by_numbers/train/12...


In [5]:
portraits_path = image_paths.merge(portraits, on = 'new_filename', how = 'inner')
portraits_path.shape

(12926, 15)

# Object detection:


In [6]:
module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1" # model ssd+mobilenet V2: small and fast.
#module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" # model FasterRCNN+InceptionResNet V2: higher accuracy,
detector = hub.load(module_handle).signatures['default']



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
2022-04-12 14:23:48.243645: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-12 14:23:48.244503: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:04:00.0 name: NVIDIA GeForce GTX TITAN X computeCapability: 5.2
coreClock: 1.076GHz coreCount: 24 deviceMemorySize: 11.93GiB deviceMemoryBandwidth: 313.37GiB/s
2022-04-12 14:23:48.245314: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:83:00.0 name: NVIDIA GeForce GTX TITAN X computeCapability: 5.2
coreClock: 1.076GHz coreCount: 24 deviceMemorySize: 11.93GiB deviceMemoryBandwidth: 313.37GiB/s
2022-04-12 14:23:48.245365: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-04-12 14:23:48.245428: I tensorflow/st

In [25]:
IMAGE_RESIZE_SHAPE = 2* [256]

def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)


def load_and_resize_image(path, new_width=IMAGE_RESIZE_SHAPE[0], new_height=IMAGE_RESIZE_SHAPE[1],
                              display=False):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  pil_image = Image.open(path)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  print("Image downloaded to %s." % filename)
  if display:
    display_image(pil_image)
  return filename


def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
  """Adds a bounding box to an image."""
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin


def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
  """Overlay labeled boxes on an image with formatted scores and label names."""
  colors = list(ImageColor.colormap.values())

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    #print("Font not found, using default font.")
    font = ImageFont.load_default()

  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[hash(class_names[i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      np.copyto(image, np.array(image_pil))
  return image, image_pil

In [26]:
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

In [27]:
def format_detection_results(result, name):
    formatted_result =  result.copy()
    for k,v in result.items():
        formatted_result
    # dict_keys(['detection_class_entities', 'detection_scores', 'detection_boxes', 'detection_class_names', 'detection_class_labels'])
    for i, col in enumerate(["x0", "y0", "x1", "y1"]):
        formatted_result[col] = [box[i] for box in formatted_result["detection_boxes"]]
    del formatted_result["detection_boxes"]
    del formatted_result["detection_class_names"]
    dtf = pd.DataFrame(formatted_result)
    dtf["new_filename"] = name
    return dtf



In [28]:
def run_detector(detector, path, name, display=True, verbose=False):
  img = load_img(path)

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  start_time = time.time()
  result = detector(converted_img)
  end_time = time.time()

  result = {key:value.numpy() for key,value in result.items()}

  if verbose: 
    print("Found %d objects." % len(result["detection_scores"]))
    print("Inference time: ", end_time-start_time)

  image_with_boxes, image_with_boxes_pil = draw_boxes(
      img.numpy(), result["detection_boxes"],
      result["detection_class_entities"], result["detection_scores"])

  if display:
    display_image(image_with_boxes)
  return image_with_boxes_pil, format_detection_results(result, name)

In [29]:
def detect_save_objects_and_image_with_box(detector, image_path):
    name = image_path.split("/")[-1]
    image_with_boxes_pil, results = run_detector(detector, image_path, name, display=False)
    image_with_boxes_pil.save(path.join(images_with_boxes_folder, name), format="JPEG", quality=90)
    results.to_csv(object_detection_results_path.replace("XXX", name))
    return results

In [30]:
#for image_path in image_paths.path:
#    run_detector(detector, image_path)

In [31]:
portraits_path.path[0].split("/")[-1]

'50598.jpg'

In [32]:
# resize image: might be useful with bigger model if it is too slow.

#resized_image_path = load_and_resize_image(portraits_path.path[0])
#resized_image = load_img(resized_image_path)
#display_image(resized_image)

In [33]:
# run model on single image and save it as test.jpg
#image_with_boxes, result = run_detector(detector, portraits_path.path[0], portraits_path.path[0].split("/")[-1])
#image_with_boxes.save("test.jpg", format="JPEG", quality=90)

In [47]:
#for i, img_path in enumerate(portraits_path.path[0:15]):
#    print(f"\n\niteration {i}\n-----------------")
#    detect_save_objects_and_image_with_box(detector, img_path)

results_per_image = [
    detect_save_objects_and_image_with_box(detector, img_path)
    for img_path in tqdm(portraits_path.path[200:2000])
]



  0%|          | 0/1800 [00:00<?, ?it/s]

In [41]:
# ,detection_class_labels,detection_class_entities,detection_scores,x0,y0,x1,y1,new_filename
relevant_columns = ["detection_class_labels", "detection_class_entities", "detection_scores", "new_filename"]

def unique_detection_classes(result_dtf):
    return result_dtf[relevant_columns].groupby(
        ['detection_class_labels', "detection_class_entities", "new_filename"], as_index=False).agg(
        {'detection_scores':max}
    )

In [42]:
global_uresults = pd.concat([unique_detection_classes(res) for res in results_per_image])

In [43]:
global_uresults

,detection_class_labels,detection_class_entities,new_filename,detection_scores
0,69,b'Person',50598.jpg,0.294567
1,308,b'Man',50598.jpg,0.080810
2,433,b'Clothing',50598.jpg,0.190289
3,434,b'Footwear',50598.jpg,0.091691
4,502,b'Human face',50598.jpg,0.177448
...,...,...,...,...
6,409,b'Wheel',74337.jpg,0.096547
7,433,b'Clothing',74337.jpg,0.413291
8,456,b'Flower',74337.jpg,0.123928
9,502,b'Human face',74337.jpg,0.165232
